In [63]:
import pandas as pd
import numpy as np
import math 

inputs wind and direction, outputs adjusted azimuth & distance<br>
given impact point of wind, calculate implied wind distance & azimuth<br>
calculate length of third side given TSG angle and lengths of the two other sides (TSG = Target, Spotter, Gunner)

In [64]:
def findTSGAngle(spotterToTargetAzimuth, spotterToGunAzimuth):
    azimuthArray = [spotterToTargetAzimuth, spotterToGunAzimuth]
    aziMin = min(azimuthArray)
    aziMax = max(azimuthArray)    


    if (360 - aziMax - aziMin) > 0:
        return min(aziMax - aziMin, 360 - aziMax - aziMin)
    else:
        return (aziMax - aziMin)
    

# print(findTSGAngle(0,90))
# print(findTSGAngle(264,171))


In [65]:
def findDistanceGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance):
    dST = spotterToTargetDistance
    dSG = spotterToGunDistance
    aTSG = findTSGAngle(spotterToTargetAzimuth,spotterToGunAzimuth)

    # Law of Cosines to solve 3rd unknown angle
    # a^2 = b^2 + c^2 − 2*b*c*cos(A)

    # math.radians() inputs degrees, outputs radians
    # math.cos() inputs radians, outputs degrees?
    # math.degrees inputs radians, outputs degrees


    distGunToTarget = math.sqrt(dST**2 + dSG**2 - 2*dST*dSG*math.cos(math.radians(aTSG)))

    return distGunToTarget

# print(findDistanceGunToTarget(253, 66, 173, 16))

In [66]:
def findTGSAngle(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance):
    # https://courses.lumenlearning.com/suny-osalgebratrig/chapter/non-right-triangles-law-of-cosines/ intuition
    # reference figure 3
    # distance GT = a
    # distance ST = b
    # distance SG = c
    # angle TSG = alpha
    # objective: find angle Beta
    # use equation cos(beta) = (a**2 + c**2 - b**2)/(2*a*c)

    dGT = findDistanceGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)
    dST = spotterToTargetDistance
    dSG = spotterToGunDistance

    # math.acos() inputs radians & outputs radians
    aTGS = math.degrees(math.acos((dGT**2 + dSG**2 - dST**2)/(2*dGT*dSG)))
    return aTGS

# print(findTGSAngle(0,3,90,4))


In [67]:
def findAzimuthGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance):
    aTGS = findTGSAngle(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)
    aTSG = findTSGAngle(spotterToTargetAzimuth, spotterToGunAzimuth)
    aSTG = 180 - (aTGS + aTSG)
    aziSG = spotterToGunAzimuth
    aziST = spotterToTargetAzimuth


    # Create imaginary box around 3 points (Spotter-Target-Gun) box & try to use alternate interior angle theorem

    # Graph 1
    # if aziSG > 180, take aziSG - 180
    # SG > ST
    # ST < 180
    # SG > 180
    # (aziSG - 180) + aTGS

    # Graph 2
    # if aziSG < 180, take aziSG + 180
    # SG > ST
    # ST < 180
    # SG < 180
    # (aziSG + 180) + aTGS

    # Graph 3
    # SG < ST
    # ST > 180
    # SG > 180
    # (aziSG - 180) - aTGS

    # Graph 4
    # SG < ST
    # ST < 180
    # SG < 180
    # same formula as graph 2
    # (aziSG + 180) - aTGS

    # if SG > 180, first term is subtraction
    # if SG < 180, first term is addition

    # if SG > ST, second term is addition
    # if SG < ST, second term is subtraction
    if (aziSG > aziST) and (aziSG > 180):
        return (aziSG - 180) + aTGS

    elif (aziSG > aziST) and (aziSG < 180):
        return (aziSG + 180) + aTGS
    
    elif (aziSG < aziST) and (aziSG > 180):
        return (aziSG - 180) - aTGS

    elif (aziSG < aziST) and (aziSG < 180):
        return (aziSG + 180) - aTGS
        
# print(findAzimuthGunToTarget(253, 66, 173, 16))

In [68]:
def comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce):
    unadjustedGunToTargetDistance = findDistanceGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)
    unadjustedGunToTargetAzimuth = findAzimuthGunToTarget(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance)

    # for when I figure out model
    # return findWindAdjustedGunToTargetAziDist(unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce)
    return [unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance]

# print(findDistanceGunToTarget(254, 65, 173, 17))
# print(findAzimuthGunToTarget(254, 65, 173, 17)) ************ test here

In [69]:
# The plan
# Purpose: build model to predict gun azi/dist given gunAzimuth, gunDistance, windAzimuth, & windForce (multi-variate?)
# Data to work with: spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, spotterToImpactAzimuth, spotterToImpactDistance, windAzimuth, windForce

# Step 0: get data from training island by firing artillery shills, adapt artilleryContinuous in order to generate data (perhaps read & write from excel file function?)

# Step 1: make model to predict impact azimuth and impact distance
# implementation of step 1:
    # model 1: gunToImpactAzimuth:
        # dependent variables: gunToTargetDistance, windAzimuth, windForce
        # independent variable: gunToImpactAzimuth

    # model 2: gunToImpactDistance:
        # dependent variables: gunToTargetDistance, windAzimuth, windForce
        # independent variable: gunToImpactDistance


# Step 2: use goal seek to find necessary gunToTargetAzimuth and gunToTargetDistance to make impact azimuth and impact distance equal to target azimuth and target distance, respectively
# implemention of step 2:
    # model to calculate newGunToTargetAzimuth to hit target
    # Input:
    # Output: the necessary azimuth in order to get impact = target

    # model to calculate newGunToTargetDistance to hit target
    # Input: 
    # Output: the necessary distance in order to get impact = target

# model
    # indepentdent variable =  distance from gun to target
    # dependent variable = difference between predicted impact and actual impact
    # use to find how error changes over distance fired
    # use to find given error at certain distance
    # error that cannot be explained by wind

In [70]:
# data function
# https://openpyxl.readthedocs.io/en/stable/

from os.path import exists
from openpyxl import Workbook
import pandas as pd


# excelFileName is the file name to save artillery data to
# caliberOfWeapon is for example "120mm" and is the name of the worksheet in the excel file
def getDataForArtilleryModels(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, excelFileName, caliberOfWeapon):
    firingHistoryArray = pd.DataFrame(columns=list(["spotterToTargetAzimuth", "spotterToTargetDistance", "spotterToGunAzimuth", "spotterToGunDistance", 
        "windAzimuth", "windForce", "unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance", "gunToImpactAzimuth", "gunToImpactDistance"]))
    print("Keep in mind multiple input responses should be space separated.")

    while True:
        print(firingHistoryArray.tail(2))
        print("\n\n")
        userInput = input("Enter <S> to stop, <C> to change variables, or press <Esc> key to continue. \t").lower() # potentially add another option to change values in dataframe if a mistake was entered

        # saves current data in dataframe to excel file & closes program
        if userInput == "s":
            # if .xlsx file already exists
            if (exists(f"{excelFileName}.xlsx")):
                # add firingHistoryArray dataframe to excel file to save data
                # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html   pd.ExcelWriter section
                # appends firingHistoryArray to existing excel file named whatever the <excelFilName> was, MUST BE IN SAME DIRECTORY AS PROGRAM

                # import excel file, converts to pandas dataframe
                xlToPandas = pd.ExcelFile(f"{excelFileName}.xlsx")
                dataFrameFromPastExcel = xlToPandas.parse(f"{caliberOfWeapon}")
                dataFrameFromPastExcel.reset_index(drop=True, inplace=True)

                # makes the index of the two merge and go from 0 to N, vs restarting from zero
                dataFrameFromPastExcel.reset_index(drop=True, inplace=True)
                firingHistoryArray.reset_index(drop=True, inplace=True)

                # merge historical excel data with new data to upload to Excel file
                combinedDataframe = pd.concat([dataFrameFromPastExcel, firingHistoryArray], ignore_index=True)
                combinedDataframe.drop(combinedDataframe.columns[combinedDataframe.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
                with pd.ExcelWriter(f"{excelFileName}.xlsx", mode = "a", engine="openpyxl", if_sheet_exists="replace") as writer:
                    combinedDataframe.to_excel(writer, sheet_name = f"{caliberOfWeapon}", header="None")
            else:
                # create new .xlsx file and save firingHistoryArray dataframe to it and save file if doesn't already exist
                # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html
                firingHistoryArray.to_excel(f"{excelFileName}.xlsx", sheet_name=caliberOfWeapon)
            print("Exiting..")
            break



        # changes inputs if inputs change, ask user if these are the correct inputs before applying
        elif (userInput == "c"):
            # display possible inputs
            # changes inputs to what the other puts in
            while True:
                currentVariableValuesArray = [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce]
                print(currentVariableValuesArray)
                variableNumbers = input("Enter numbers 1 through 6 to change: spotterToTargetAzimuth(1), spotterToTargetDistance(2), spotterToGunAzimuth(3), spotterToGunDistance(4), windAzimuth(5), windForce(6) Press <Esc> to leave.").split()
                if variableNumbers == "":
                    break

                # converts variables to int
                variableNumbers = [int(x) for x in variableNumbers]

                columnNames = firingHistoryArray.columns

                # prints out variables that can be changed 
                for x in variableNumbers:
                    print(f"{columnNames[x-1]}",end=" ")
                print("These are the respective values to change.")

                
                variableValues = input("Enter the values to change.").split()
                variableValues = [int(x) for x in variableValues]

                # assigns new values to existing array
                for i in range(len(variableNumbers)):
                    currentVariableValuesArray[variableNumbers[i]-1] = variableValues[i]

                # assigns values in array to variables 
                spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce = currentVariableValuesArray

                # confirm these are the correct variables?
                print(currentVariableValuesArray)
                userInputToLeaveChange = input("Are these the correct variables? Enter <Y> to leave. \t").lower()
                if userInputToLeaveChange == "y":
                    break

        # once impact hits, ask for impact azimuth and impact distance
        while True:
            impactAzi, impactDist = input("Please enter impact azimuth & impact distance, i.e. spotterToImpactAzimuth & spotterToImpactDistance \t").split()
            print(f"Impact Azimuth: {impactAzi} \n Impact Distance: {impactDist}")
            areImpactCorrectInput = input("Are these impact values correct? Enter <Y> to leave loop.").lower()
            if areImpactCorrectInput == "y":
                break

        impactValuesSpotterToImpact = [int(impactAzi), int(impactDist)]
        impactValuesGunToImpact = [findAzimuthGunToTarget(impactValuesSpotterToImpact[0], impactValuesSpotterToImpact[1], spotterToGunAzimuth, spotterToGunDistance), findDistanceGunToTarget(impactValuesSpotterToImpact[0], impactValuesSpotterToImpact[1], spotterToGunAzimuth, spotterToGunDistance)]
        
        # for when I have a model to calculate adjustments for wind
        # gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        # firingHistoryArray = firingHistoryArray.append([spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals + impactValuesGunToImpact)

        gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        valuesToStoreInDataframe = [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals + impactValuesGunToImpact
        firingHistoryArrayLength = len(firingHistoryArray)
        firingHistoryArray.reset_index(drop=True, inplace=True)
        firingHistoryArray.loc[firingHistoryArrayLength] = valuesToStoreInDataframe


# input variables
# (spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, excelFileName, caliberOfWeapon)
# NOTE: cannot have existing Excel file open when saving to it
# getDataForArtilleryModels(254,65,173,17,124,4,"artilleryData","120mm")


In [71]:
# findMultiLinearregression 
# https://www.w3schools.com/python/python_ml_multiple_regression.asp
# multilinear regression model
# caliberOfweapon is worksheet name
# model 1: gunToImpactAzimuth:
    # dependent variables: gunToTargetDistance, windAzimuth, windForce
    # independent variable: gunToImpactAzimuth

# model 2: gunToImpactDistance:
    # dependent variables: gunToTargetDistance, windAzimuth, windForce
    # independent variable: gunToImpactDistance
    
from sklearn import linear_model

def findMultilinearRegression(excelFileName, caliberOfWeapon, indVars, depVar):
    xlToPandas = pd.ExcelFile(f"{excelFileName}.xlsx")
    df = xlToPandas.parse(f"{caliberOfWeapon}")
    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    independentVars = df[indVars]
    dependentVar = df[depVar]
    regr = linear_model.LinearRegression()
    regr.fit(independentVars.values, dependentVar)
    return regr


gunToImpactAzimuthRegression = findMultilinearRegression("artilleryData", "120mm", ["unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance","windAzimuth","windForce"],"gunToImpactAzimuth")
gunToImpactDistanceRegression = findMultilinearRegression("artilleryData", "120mm",["unadjustedGunToTargetDistance","windAzimuth","windForce"],"gunToImpactDistance")

# https://www.geeksforgeeks.org/saving-a-machine-learning-model/
# how to save the model instead of retraining 
# 

In [72]:
# predictedgGunToImpactAzimuth = gunToImpactAzimuthRegression.predict([[300,200,300,4]])
# predictedGunToImpactDistance = gunToImpactDistanceRegression.predict([[200,300,4]])
# print(predictedgGunToImpactAzimuth[0])
# print(predictedGunToImpactDistance[0])

In [73]:
# scipy.optimize wind adjustments
import numpy as np
from scipy.optimize import minimize
# https://youtu.be/M7ZA9fq2zCE
# learn scipy.optimize, rewrite all functions below
# https://youtu.be/M7ZA9fq2zCE

# NOTE: DEPENDENDENT ON REGRESSIONS ABOVE
# https://www.geeksforgeeks.org/saving-a-machine-learning-model/
# how to save the model instead of retraining 

def predictedGunToTargetAziDist(unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce, adjustedGunToImpactAzimuthRegression, adjustedGunToImpactDistanceRegression):
    predictedgGunToImpactAzimuth = adjustedGunToImpactAzimuthRegression.predict([[unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce]])
    predictedGunToImpactDistance = adjustedGunToImpactDistanceRegression.predict([[unadjustedGunToTargetDistance, windAzimuth, windForce]])
    return [predictedgGunToImpactAzimuth, predictedGunToImpactDistance]

def adjustForWindGunToTargetAziDist(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, adjustedGunToImpactAzimuthRegression, adjustedGunToImpactDistanceRegression):
    unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
    testGunToTargetAzimuth, testGunToTargetDistance = unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance
    x0 = [testGunToTargetAzimuth, testGunToTargetDistance]

    # idea change testGunToTargetAzimuth and testGunToTargetDistance, hold all other variables constant (i.e. equality constraint)
    # minimize objective_fcn
    def objectiveFunction(x):
        testGunToTargetAzimuth, testGunToTargetDistance = x
        predictedValues = predictedGunToTargetAziDist(testGunToTargetAzimuth, testGunToTargetDistance, windAzimuth, windForce, adjustedGunToImpactAzimuthRegression, adjustedGunToImpactDistanceRegression)
        return abs(predictedValues[0] - unadjustedGunToTargetAzimuth) + abs(predictedValues[1] - unadjustedGunToTargetDistance)

    # objective find GunToTargetAzimuth and GunToTargetDistance that minimize (predictedgGunToImpactAzimuth - unadjustedGunToTargetAzimuth) and (predictedGunToImpactDistance - unadjustedGunToTargetDistance)
    result = minimize(objectiveFunction, x0, method = "SLSQP")

    # returns [adjustedGunToTargetAzimuth, adjustedGunToTargetDistance] necessary in order for impact to hit target
    return [result["x"][0], result["x"][1]]



        
        

# print(adjustForWindGunToTargetAziDist(300, 50, 350, 5, 50, 5, gunToImpactAzimuthRegression, gunToImpactDistanceRegression))

In [74]:
# spotterArtillery function
# next task: adapt data gathering function into combat function while incorporating adjustForWindGunToTargetAziDist function that uses scipy.optimize
# another task: run regressions with actual data to see what they predict & compare to actual impact values
# https://openpyxl.readthedocs.io/en/stable/

from os.path import exists
from openpyxl import Workbook
import pandas as pd

# excelFileName is the file name which is used to create the regression to model to calculate adjustments, i.e. where the artillery data is stored
# caliberOfWeapon is the worksheet name, e.g. "120mm"
def spotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, excelFileName, caliberOfWeapon):
    firingHistoryArray = pd.DataFrame(columns=list(["spotterToTargetAzimuth", "spotterToTargetDistance", "spotterToGunAzimuth", "spotterToGunDistance", 
        "windAzimuth", "windForce", "unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance", "adjustedGunToTargetAzimuth", "adjustedGunToTargetDistance"]))
    print("Keep in mind multiple input responses should be space separated.")
    
    adjustedGunToImpactAzimuthRegression = findMultilinearRegression(excelFileName, caliberOfWeapon, ["unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance","windAzimuth","windForce"],"gunToImpactAzimuth")
    adjustedGunToImpactDistanceRegression = findMultilinearRegression(excelFileName, caliberOfWeapon,["unadjustedGunToTargetDistance","windAzimuth","windForce"],"gunToImpactDistance")

    while True:
        print(firingHistoryArray.tail(2))
        print("\n\n")
        userInput = input("Enter <S> to stop, <C> to change variables, or press <Esc> key to continue. \t").lower() # potentially add another option to change values in dataframe if a mistake was entered

        # saves current data in dataframe to excel file & closes program
        if userInput == "s":
            print("Exiting..")
            break

        # changes inputs if inputs change, ask user if these are the correct inputs before applying
        elif (userInput == "c"):
            # display possible inputs
            # changes inputs to what the other puts in
            while True:
                currentVariableValuesArray = [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce]
                print(f"Enter numbers 1 through 6 to change: \n spotterToTargetAzimuth(1), spotterToTargetDistance(2), spotterToGunAzimuth(3), spotterToGunDistance(4), windAzimuth(5), windForce(6) \n {currentVariableValuesArray} \n")
                variableNumbers = input("Enter numbers 1 through 6 to change: spotterToTargetAzimuth(1), spotterToTargetDistance(2), spotterToGunAzimuth(3), spotterToGunDistance(4), windAzimuth(5), windForce(6) Press <Esc> to leave.").split()
                if variableNumbers == "":
                    break

                # converts variables to int
                variableNumbers = [int(x) for x in variableNumbers]

                columnNames = firingHistoryArray.columns

                # prints out variables that can be changed 
                for x in variableNumbers:
                    print(f"{columnNames[x-1]}",end=" ")
                print("\n These are the respective values to change. \n")

                
                variableValues = input("Enter the values to change.").split()
                variableValues = [int(x) for x in variableValues]

                # assigns new values to existing array
                for i in range(len(variableNumbers)):
                    currentVariableValuesArray[variableNumbers[i]-1] = variableValues[i]

                # assigns values in array to variables 
                spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce = currentVariableValuesArray

                # confirm these are the correct variables?
                print(f"Are these the correct variables? Enter <Y> to leave. \n{currentVariableValuesArray} \n")
                userInputToLeaveChange = input("Are these the correct variables? Enter <Y> to leave.").lower()
                if userInputToLeaveChange == "y":
                    break

    
        # for when I have a model to calculate adjustments for wind
        # gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        # firingHistoryArray = firingHistoryArray.append([spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals + impactValuesGunToImpact)

        gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        windAdjustedGunToTargetAziDist = adjustForWindGunToTargetAziDist(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce, adjustedGunToImpactAzimuthRegression, adjustedGunToImpactDistanceRegression)

        print(f"Wind Adjusted Gun To Target Azimuth: {windAdjustedGunToTargetAziDist[0]}")
        print(f"Wind Adjusted Gun To Target Distance: {windAdjustedGunToTargetAziDist[1]} \n")

        valuesToStoreInDataframe = [spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals + windAdjustedGunToTargetAziDist
        firingHistoryArrayLength = len(firingHistoryArray)
        firingHistoryArray.reset_index(drop=True, inplace=True)
        firingHistoryArray.loc[firingHistoryArrayLength] = valuesToStoreInDataframe


# input variables
# (spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
# NOTE: cannot have existing Excel file open when saving to it
# spotterArtillery(254,65,173,17,124,4,"artilleryData","120mm")


In [76]:
# noSpotterArtillery function
# next task: adapt data gathering function into combat function while incorporating adjustForWindGunToTargetAziDist function that uses scipy.optimize
# another task: run regressions with actual data to see what they predict & compare to actual impact values
# https://openpyxl.readthedocs.io/en/stable/

from os.path import exists
from openpyxl import Workbook
import pandas as pd

# excelFileName is the file name which is used to create the regression to model to calculate adjustments, i.e. where the artillery data is stored
# caliberOfWeapon is the worksheet name, e.g. "120mm"
def noSpotterArtillery(unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce, excelFileName, caliberOfWeapon):
    firingHistoryArray = pd.DataFrame(columns=list(["unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance", "windAzimuth", "windForce", "adjustedGunToTargetAzimuth", "adjustedGunToTargetDistance"]))
    print("Keep in mind multiple input responses should be space separated.")
    
    adjustedGunToImpactAzimuthRegression = findMultilinearRegression(excelFileName, caliberOfWeapon, ["unadjustedGunToTargetAzimuth", "unadjustedGunToTargetDistance","windAzimuth","windForce"],"gunToImpactAzimuth")
    adjustedGunToImpactDistanceRegression = findMultilinearRegression(excelFileName, caliberOfWeapon,["unadjustedGunToTargetDistance","windAzimuth","windForce"],"gunToImpactDistance")

    while True:
        print(firingHistoryArray.tail(2))
        print("\n\n")
        userInput = input("Enter <S> to stop, <C> to change variables, or press <Esc> key to continue. \t").lower() # potentially add another option to change values in dataframe if a mistake was entered

        # saves current data in dataframe to excel file & closes program
        if userInput == "s":
            print("Exiting..")
            break

        # changes inputs if inputs change, ask user if these are the correct inputs before applying
        elif (userInput == "c"):
            # display possible inputs
            # changes inputs to what the other puts in
            while True:
                currentVariableValuesArray = [unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce]
                print(f"Enter numbers 1 through 4 to change: \n unadjustedGunToTargetAzimuth(1), unadjustedGunToTargetDistance(2), windAzimuth(3), windForce(4) \n {currentVariableValuesArray} \n")
                variableNumbers = input("Enter numbers 1 through 4 to change: unadjustedGunToTargetAzimuth(1), unadjustedGunToTargetDistance(2), windAzimuth(3), windForce(4) Press <Esc> to leave.").split()
                if variableNumbers == "":
                    break

                # converts variables to int
                variableNumbers = [int(x) for x in variableNumbers]

                columnNames = firingHistoryArray.columns

                # prints out variables that can be changed 
                for x in variableNumbers:
                    print(f"{columnNames[x-1]}",end=" ")
                print("\n These are the respective values to change. \n")

                
                variableValues = input("Enter the values to change.").split()
                variableValues = [int(x) for x in variableValues]

                # assigns new values to existing array
                for i in range(len(variableNumbers)):
                    currentVariableValuesArray[variableNumbers[i]-1] = variableValues[i]

                # assigns values in array to variables 
                unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce = currentVariableValuesArray

                # confirm these are the correct variables?
                print(f"Are these the correct variables? Enter <Y> to leave. \n{currentVariableValuesArray} \n")
                userInputToLeaveChange = input("Are these the correct variables? Enter <Y> to leave.").lower()
                if userInputToLeaveChange == "y":
                    break

    
        # for when I have a model to calculate adjustments for wind
        # gunToTargetNewVals = comprehensiveSpotterArtillery(spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce)
        # firingHistoryArray = firingHistoryArray.append([spotterToTargetAzimuth, spotterToTargetDistance, spotterToGunAzimuth, spotterToGunDistance, windAzimuth, windForce] + gunToTargetNewVals + impactValuesGunToImpact)

        def noSpotterAdjustForWindGunToTargetAziDist(unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce, adjustedGunToImpactAzimuthRegression, adjustedGunToImpactDistanceRegression):
            testGunToTargetAzimuth, testGunToTargetDistance = unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance
            x0 = [testGunToTargetAzimuth, testGunToTargetDistance]

            # idea change testGunToTargetAzimuth and testGunToTargetDistance, hold all other variables constant (i.e. equality constraint)
            # minimize objective_fcn
            def objectiveFunction(x):
                testGunToTargetAzimuth, testGunToTargetDistance = x
                predictedValues = predictedGunToTargetAziDist(testGunToTargetAzimuth, testGunToTargetDistance, windAzimuth, windForce, adjustedGunToImpactAzimuthRegression, adjustedGunToImpactDistanceRegression)
                return abs(predictedValues[0] - unadjustedGunToTargetAzimuth) + abs(predictedValues[1] - unadjustedGunToTargetDistance)

            # objective find GunToTargetAzimuth and GunToTargetDistance that minimize (predictedgGunToImpactAzimuth - unadjustedGunToTargetAzimuth) and (predictedGunToImpactDistance - unadjustedGunToTargetDistance)
            result = minimize(objectiveFunction, x0, method = "SLSQP")

            # returns [adjustedGunToTargetAzimuth, adjustedGunToTargetDistance] necessary in order for impact to hit target
            return [result["x"][0], result["x"][1]] 

        windAdjustedGunToTargetAziDist = noSpotterAdjustForWindGunToTargetAziDist(unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce, adjustedGunToImpactAzimuthRegression, adjustedGunToImpactDistanceRegression)

        print(f"Wind Adjusted Gun To Target Azimuth: {windAdjustedGunToTargetAziDist[0]}")
        print(f"Wind Adjusted Gun To Target Distance: {windAdjustedGunToTargetAziDist[1]} \n")

        valuesToStoreInDataframe = [unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce] + windAdjustedGunToTargetAziDist
        firingHistoryArrayLength = len(firingHistoryArray)
        firingHistoryArray.reset_index(drop=True, inplace=True)
        firingHistoryArray.loc[firingHistoryArrayLength] = valuesToStoreInDataframe


# input variables
# (unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce)
# NOTE: cannot have existing Excel file open when saving to it
noSpotterArtillery(254,65,124,4,"artilleryData","120mm")


Keep in mind multiple input responses should be space separated.
Empty DataFrame
Columns: [unadjustedGunToTargetAzimuth, unadjustedGunToTargetDistance, windAzimuth, windForce, adjustedGunToTargetAzimuth, adjustedGunToTargetDistance]
Index: []



Enter numbers 1 through 4 to change: 
 unadjustedGunToTargetAzimuth(1), unadjustedGunToTargetDistance(2), windAzimuth(3), windForce(4) 
 [254, 65, 124, 4] 

unadjustedGunToTargetAzimuth unadjustedGunToTargetDistance 
 These are the respective values to change. 

Are these the correct variables? Enter <Y> to leave. 
[250, 50000, 124, 4] 

Wind Adjusted Gun To Target Azimuth: 289.2239378827087
Wind Adjusted Gun To Target Distance: -39839.68136805467 

   unadjustedGunToTargetAzimuth  unadjustedGunToTargetDistance  windAzimuth  \
0                         250.0                        50000.0        124.0   

   windForce  adjustedGunToTargetAzimuth  adjustedGunToTargetDistance  
0        4.0                  289.223938                -39839.681368